-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Fast Changing Facts with Stream-Static Join

In this lesson, you'll join streaming heart rate data with the completed workouts table.

We'll be creating the table `workout_bpm` in our architectural diagram.

<img src="https://files.training.databricks.com/images/ade/ADE_arch_workout_bpm.png" width="60%" />

## Learning Objectives
By the end of this lesson, students will be able to:
- Describe guarantees around versioning and matching for stream-static joins
- Leverage Spark SQL and PySpark to process stream-static joins

In [0]:
%run ../Includes/join-setup

A helper function has been defined to process a new batch of data to the source tables used in this lesson. (Note: this may take around 2 minutes.)

In [0]:
process_join_sources()

Set up your streaming temp view. Note that we will only be streaming from **one** of our tables. The `completed_workouts` table is no longer streamable as it breaks the requirement of an ever-appending source for Structured Streaming. However, when performing a stream-static join with a Delta table, each batch will confirm that the newest version of the static Delta table is being used.

In [0]:
spark.readStream.table("heart_rate_silver").createOrReplaceTempView("TEMP_heart_rate_silver")

## Perform Stream-Static Join to Align Workouts to Heart Rate Recordings

Below we'll configure our query to join our stream to our `completed_workouts` table.

Note that our heart rate recordings only have `device_id`, while our workouts use `user_id` as the unique identifier. We'll need to use our `user_lookup` table to match these values.

Importantly, our devices occasionally send messages with negative recordings, which represent a potential error in the recorded values. We'll need to define predicate conditions to ensure that only positive recordings are processed.

In [0]:
spark.sql("""
  SELECT d.user_id, d.workout_id, d.session_id, time, heartrate
  FROM TEMP_heart_rate_silver c
  INNER JOIN (
    SELECT a.user_id, b.device_id, workout_id, session_id, start_time, end_time
    FROM completed_workouts a
    INNER JOIN user_lookup b
    ON a.user_id = b.user_id) d
  ON c.device_id = d.device_id AND time BETWEEN start_time AND end_time
  WHERE c.bpm_check = 'OK'""").createOrReplaceTempView("TEMP_workout_bpm")

### Write Stream in Append Mode

Below, we'll use our streaming temp view from above to insert new values into our `workout_bpm` table.

Explore this results table below.

In [0]:
%sql

SELECT COUNT(*)
FROM workout_bpm

In [0]:
%sql

SELECT * FROM workout_bpm

If desired, process another batch through all tables and update these results.

In [0]:
process_join_sources()
process_workout_bpm()

In [0]:
%sql

SELECT COUNT(*)
FROM workout_bpm

-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>